In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix, f1_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input, Dropout, BatchNormalization

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
import joblib
from tensorflow.keras.models import load_model

# 1 - Load dataset

In [15]:
data = pd.read_csv('./Data/final_dataset.csv')
data.head()

,id,seen_by_model,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label,createdAt,updatedAt,deletedAt
0,9470,1,0.441011,0.098403,0.308838,0.010569,3328.124539,8.107549e+05,3382.691560,114583.284474,...,-0.968601,87.470032,2.991356,59.765133,-1.670792,59.670425,7,2021-11-22 17:52:43,2021-11-22 17:52:43,NaN
1,9471,1,0.370265,0.083063,0.171207,0.006743,2915.715009,1.556398e+06,2599.689672,324970.562618,...,5.818873,138.011856,4.596396,144.546539,1.729125,211.818741,7,2021-11-22 17:52:43,2021-11-22 17:52:43,NaN
2,9472,1,0.385647,0.071537,0.229402,0.015564,3880.270183,2.722230e+06,3186.927682,254165.389259,...,2.303481,64.394051,0.785539,59.500866,-2.959064,73.443031,7,2021-11-22 17:52:43,2021-11-22 17:52:43,NaN
3,9473,1,0.407606,0.107868,0.079086,0.000538,2271.813617,6.391978e+05,2707.634095,187545.070021,...,-3.473018,122.847267,-2.771165,68.308220,-0.633971,63.867836,7,2021-11-22 17:52:43,2021-11-22 17:52:43,NaN
4,9474,1,0.397764,0.072330,0.242004,0.005612,2911.060481,5.488350e+05,2956.118960,145420.851816,...,3.052081,117.361862,-1.915870,47.333481,2.078118,60.737377,7,2021-11-22 17:52:43,2021-11-22 17:52:43,NaN


In [49]:
data.columns

Index(['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'tempo', 'mfcc1_mean', 'mfcc1_var',
       'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean',
       'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc6_var',
       'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean',
       'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean', 'mfcc11_var',
       'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean',
       'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var',
       'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var', 'mfcc19_mean',
       'mfcc19_var', 'mfcc20_mean', 'mfcc20_var', 'label'],
      dtype='object')

In [17]:
drop_layers = ['id', 'seen_by_model', 'createdAt', 'updatedAt', 'deletedAt']
data = data.drop(columns=drop_layers)
data.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.441011,0.098403,0.308838,0.010569,3328.124539,8.107549e+05,3382.691560,114583.284474,7897.323031,1.917331e+06,...,42.023411,-5.012711,66.960938,-0.968601,87.470032,2.991356,59.765133,-1.670792,59.670425,7
1,0.370265,0.083063,0.171207,0.006743,2915.715009,1.556398e+06,2599.689672,324970.562618,5661.195497,5.154728e+06,...,94.394127,1.313031,144.447861,5.818873,138.011856,4.596396,144.546539,1.729125,211.818741,7
2,0.385647,0.071537,0.229402,0.015564,3880.270183,2.722230e+06,3186.927682,254165.389259,7703.815192,5.101757e+06,...,89.115379,5.156198,82.118690,2.303481,64.394051,0.785539,59.500866,-2.959064,73.443031,7
3,0.407606,0.107868,0.079086,0.000538,2271.813617,6.391978e+05,2707.634095,187545.070021,5239.177972,3.272871e+06,...,58.049915,-11.866402,100.287689,-3.473018,122.847267,-2.771165,68.308220,-0.633971,63.867836,7
4,0.397764,0.072330,0.242004,0.005612,2911.060481,5.488350e+05,2956.118960,145420.851816,6284.660711,2.669257e+06,...,50.164764,3.505230,64.743500,3.052081,117.361862,-1.915870,47.333481,2.078118,60.737377,7


In [15]:
corr = data.corr().label
d = corr[(corr < -0.05) | (corr >= 0)]
not_d_cols = list(corr[(corr > -0.1) & (corr < 0.1)].index) + ['label']
print(len(d))

51


# 2 - Pre-process data

In [16]:
data = data.sample(frac=1)

X = data.drop(columns=not_d_cols)
y = data.label

In [17]:
X.head()

,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,...,mfcc8_mean,mfcc9_mean,mfcc10_mean,mfcc11_mean,mfcc12_mean,mfcc13_var,mfcc14_mean,mfcc16_mean,mfcc18_mean,mfcc20_var
11293,0.405778,0.137611,2778.788868,1.006514e+06,2729.904294,144891.922787,5906.192902,2.679924e+06,0.122871,0.107774,...,6.060346,5.871454,6.013030,-4.361856,8.058302,100.394760,9.925915,4.862923,3.597121,79.213356
12471,0.117130,0.159906,912.852102,1.868348e+04,1249.502420,19498.686402,1625.357148,1.103608e+05,0.044290,0.042329,...,3.412008,-2.406624,-3.820136,-12.571994,-4.409249,34.084389,-0.867796,-4.911867,-3.018001,22.610453
16297,0.310476,0.077152,1508.299992,9.249421e+04,1888.571698,98090.452026,3129.508841,5.887676e+05,0.065882,0.061542,...,4.498883,-7.883408,8.132590,-7.473206,0.264852,52.306938,-3.762148,3.227815,2.629616,63.270176
4512,0.144873,0.157399,2345.659311,1.808511e+05,2285.741002,54629.904141,5062.339659,6.203109e+05,0.123477,0.108230,...,19.186220,-19.094240,11.536619,-8.293885,17.819607,58.953903,1.110843,5.375949,6.028002,71.465797
17175,0.522142,0.105582,1198.589771,2.086582e+05,1870.436167,178024.577045,2363.231572,1.942687e+06,0.039764,0.038183,...,1.919226,-7.238740,3.190215,-5.991389,2.171602,32.406166,5.710892,2.922934,5.966251,56.034161


In [19]:
count = []
tdf = pd.DataFrame(data=y, columns=['label'])

for i in range(10):
    count.append(len(tdf[tdf.label == i]))
    
count

[1450, 2994, 1396, 2423, 1919, 2237, 1529, 1475, 1550, 1201]

## 2.1 - Scale data

In [22]:
final_scaler = StandardScaler()
final_scaler.fit(X)
X_scaled = pd.DataFrame(data=final_scaler.transform(X), index=X.index, columns=X.columns)

X_scaled.head()

,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,zero_crossing_rate_mean,zero_crossing_rate_var,...,mfcc8_mean,mfcc9_mean,mfcc10_mean,mfcc11_mean,mfcc12_mean,mfcc13_var,mfcc14_mean,mfcc16_mean,mfcc18_mean,mfcc20_var
11293,0.185951,0.024260,0.802670,1.225241,0.813792,0.023571,0.822095,0.525752,0.581472,0.648594,...,-0.056145,1.354857,0.047227,-0.004411,0.686255,0.990533,1.444643,0.529465,0.389622,0.359136
12471,-1.840819,0.289567,-1.523312,-0.953860,-1.564394,-0.931789,-1.575560,-1.074266,-1.176800,-1.257116,...,-0.333932,0.285990,-1.195425,-1.238491,-1.187346,-0.705532,-0.492252,-1.374192,-0.979614,-0.908283
16297,-0.483221,-0.695175,-0.781057,-0.791038,-0.537764,-0.333005,-0.733099,-0.776371,-0.693674,-0.697647,...,-0.219929,-0.421173,0.315083,-0.472083,-0.484931,-0.239441,-1.011634,0.211025,0.189362,0.002146
4512,-1.646020,0.259733,0.262753,-0.596127,0.100267,-0.664127,0.349461,-0.756730,0.595036,0.661897,...,1.320639,-1.868717,0.745263,-0.595441,2.153166,-0.069428,-0.137192,0.629378,0.892780,0.185657
17175,1.003007,-0.356870,-1.167126,-0.534786,-0.566897,0.276006,-1.162283,0.066689,-1.278073,-1.377834,...,-0.490511,-0.337933,-0.309502,-0.249349,-0.198388,-0.748457,0.688271,0.151649,0.879998,-0.159879


In [52]:
len(X_test.columns)

NameError: name 'X_test' is not defined

## 2.2 - Detect and handle outliers

In [18]:
#outliers = {}
threshold = 3 # 3 standard deviations

for feature in data.drop(columns='label').columns:
    median = data[feature].median()
    std = data[feature].std()

    #outliers = training_set[(abs(training_set[feature] - median) / std) > threshold].index
    
    Q1 = data[feature].quantile(0.25)
    Q3 = data[feature].quantile(0.75)
    IQR = Q3 - Q1

    [MnIF, MxIF] = [Q1-1.5*IQR, Q3+1.5*IQR]
    [MnOF, MxOF] = [Q1-3*IQR, Q3+3*IQR]

    upper_outliers = data[data[feature] >= MxIF].index
    lower_outliers = data[data[feature] <= MnIF].index

    data.loc[data.index.isin(upper_outliers), feature] = data[feature].mean()
    data.loc[data.index.isin(lower_outliers), feature] = data[feature].mean()
    
    
    # Mean imputation
    #training_set.loc[training_set.index.isin(outliers), feature] = training_set[feature].mean()
    # Median imputation
    #training_set.loc[training_set.index.isin(outliers), feature] = training_set[feature].mean()
    # Mean imputation
    #training_set.loc[training_set.index.isin(outliers), feature] = training_set[feature].mean()


# 4 - Train final models

In [21]:
data = data.sample(frac=1)
data.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
16358,0.341510,0.152116,0.127034,0.002634,1044.296125,109127.412585,1712.581025,154227.981841,1758.678151,1.359509e+06,...,66.549591,-4.135895,57.389236,0.463728,74.450180,-5.197517,90.673256,-1.773638,73.464279,5
529,0.416860,0.070353,0.096615,0.000541,2596.942180,112627.812375,2276.723561,10757.431041,5149.012880,1.435360e+05,...,16.142998,-11.042757,13.387736,-0.230141,19.193823,-6.541142,19.642311,-2.406087,14.057612,6
10075,0.540842,0.149940,0.106600,0.002353,2111.305510,642785.798813,2658.174337,235831.724627,4834.953517,4.475644e+06,...,36.052967,-1.649200,36.219959,2.898467,37.318531,-5.350078,66.843880,3.300897,44.121864,0
11515,0.296642,0.125661,0.223797,0.004042,794.976860,22984.969162,1267.367853,23797.207581,1382.921259,1.800877e+05,...,31.004953,-4.917014,28.733402,0.913761,33.741051,-4.339030,28.831583,-2.450643,17.550634,1
7620,0.382525,0.054741,0.142644,0.000419,2733.518657,149015.086268,2619.606037,35443.318536,5627.235501,5.707395e+05,...,26.936996,-1.752880,27.992992,3.084649,27.824123,-4.181076,25.853157,4.474030,21.873610,6


In [43]:
X = data.drop(columns='label')
y = data.label

final_scaler = StandardScaler()
final_scaler.fit(X)
X_scaled = pd.DataFrame(data=final_scaler.transform(X), columns=X.columns)
X_scaled.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,-0.251926,2.121062,0.046310,0.606474,-1.369249,-0.863721,-0.986994,0.297976,-1.501102,-0.267290,...,-0.344878,0.700176,-0.137080,0.287625,-0.309152,0.995694,-0.686780,1.380898,-0.635965,0.569543
1,0.302926,-0.168245,-0.419021,-0.822625,0.590551,-0.853196,0.020781,-1.234232,0.398274,-1.132931,...,0.866076,-1.451952,-1.806199,-1.538964,-0.477836,-1.311167,-1.047564,-1.296220,-0.781041,-1.457629
2,1.215899,2.060123,-0.266283,0.414299,-0.022435,0.740847,0.702199,1.169471,0.222328,1.951059,...,0.512173,-0.601888,0.463857,-0.591153,0.282747,-0.554490,-0.727744,0.482781,0.528066,-0.431727
3,-0.582323,1.380328,1.526559,1.567944,-1.683947,-1.122728,-1.782317,-1.094973,-1.711613,-1.106910,...,-1.358755,-0.817415,-0.325846,-0.901935,-0.199747,-0.703844,-0.456263,-0.949882,-0.791261,-1.338435
4,0.050093,-0.605349,0.285117,-0.905848,0.762943,-0.743789,0.633301,-0.970597,0.666190,-0.828809,...,0.425072,-0.991098,0.438802,-0.932671,0.328009,-0.950866,-0.413849,-1.062137,0.797167,-1.190919


In [241]:
final_scaler.n_features_in_

35

In [53]:
len(X.columns)

55

## 4.1 - KNN

In [29]:
X.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
16358,0.341510,0.152116,0.127034,0.002634,1044.296125,109127.412585,1712.581025,154227.981841,1758.678151,1.359509e+06,...,0.704518,66.549591,-4.135895,57.389236,0.463728,74.450180,-5.197517,90.673256,-1.773638,73.464279
529,0.416860,0.070353,0.096615,0.000541,2596.942180,112627.812375,2276.723561,10757.431041,5149.012880,1.435360e+05,...,6.177681,16.142998,-11.042757,13.387736,-0.230141,19.193823,-6.541142,19.642311,-2.406087,14.057612
10075,0.540842,0.149940,0.106600,0.002353,2111.305510,642785.798813,2658.174337,235831.724627,4834.953517,4.475644e+06,...,4.578144,36.052967,-1.649200,36.219959,2.898467,37.318531,-5.350078,66.843880,3.300897,44.121864
11515,0.296642,0.125661,0.223797,0.004042,794.976860,22984.969162,1267.367853,23797.207581,1382.921259,1.800877e+05,...,-3.877915,31.004953,-4.917014,28.733402,0.913761,33.741051,-4.339030,28.831583,-2.450643,17.550634
7620,0.382525,0.054741,0.142644,0.000419,2733.518657,149015.086268,2619.606037,35443.318536,5627.235501,5.707395e+05,...,4.184471,26.936996,-1.752880,27.992992,3.084649,27.824123,-4.181076,25.853157,4.474030,21.873610


In [44]:
y.values

array([5, 6, 0, ..., 5, 3, 1])

In [45]:
y = pd.DataFrame(data=y.values, index=y.index, columns=['label'])

ValueError: 1 columns passed, passed data had 18174 columns

In [46]:
y

16358    5
529      6
10075    0
11515    1
7620     6
        ..
2831     1
9941     0
16809    5
1507     3
3008     1
Name: label, Length: 18174, dtype: int64

In [50]:
final_knn_model = KNeighborsClassifier(n_neighbors=2, weights='distance', p=1)
knn_pipeline = make_pipeline(StandardScaler(), final_knn_model)

knn_pipeline.fit(X.values, y.values)
accuracy_score(y, knn_pipeline.predict(X))

0.9996698580389568

In [51]:
# Save model
knn_model_file = "./model-server/ml-models/knn_v4.pkl"  
joblib.dump(knn_pipeline, knn_model_file)

['./model-server/ml-models/knn_v4.pkl']

## 4.2 - Neural Network

In [131]:
class NNWrapper:
    def __init__(self):
        self.model = keras.Sequential([
            Dense(600, activation='relu', input_shape=[len(X_train.columns)]),
            Dropout(0.5),
            Dense(300, activation='relu'),
            Dropout(0.5),
            Dense(128, activation='relu'),
            Dropout(0.4),
            Dense(10, activation='softmax')
        ])
        self.model.compile(metrics=['accuracy'], loss='sparse_categorical_crossentropy',
            optimizer='adam')
        
    def fit(self, X, y, X_val, y_val):
        early_stopping = EarlyStopping(
            patience=10,
            min_delta=0.001,
            restore_best_weights=True
        )
        return self.model.fit(
            X, y,
            validation_data=(X_val, y_val),
            epochs=1000,
            callbacks=[early_stopping],
            verbose=1)
    
    def predict(self, X):
        nn_prediction_distributions = self.model.predict(X)
        nn_prediction = []
        for i in range(len(nn_prediction_distributions)):
            nn_prediction.append(np.argmax(nn_prediction_distributions[i]))
            
        return nn_prediction
        

In [232]:
_X_scaled, X_val, _y_scaled, y_val = train_test_split(X_scaled, y, test_size=0.2)

nn_model = NNWrapper()

nn_model.fit(_X_scaled, _y_scaled, X_val, y_val)
nn_prediction = nn_model.predict(X_scaled)
 
#nn_pipeline = make_pipeline(StandardScaler(), nn_model)
#nn_pipeline.fit(X, y, nnwrapper__X_val=X, nnwrapper__y_val=y)
#nn_prediction = nn_pipeline.predict(X)
    
nn_acc = accuracy_score(y, nn_prediction)
print("Accuracy: " + str(nn_acc))

Epoch 1/1000
301/301 [==============================] - 2s 3ms/step - loss: 1.1853 - accuracy: 0.6020 - val_loss: 0.7841 - val_accuracy: 0.7516
Epoch 2/1000
301/301 [==============================] - 1s 2ms/step - loss: 0.8557 - accuracy: 0.7184 - val_loss: 0.6564 - val_accuracy: 0.7865
Epoch 3/1000
301/301 [==============================] - 1s 2ms/step - loss: 0.7566 - accuracy: 0.7496 - val_loss: 0.6033 - val_accuracy: 0.8077
Epoch 4/1000
301/301 [==============================] - 1s 2ms/step - loss: 0.6837 - accuracy: 0.7717 - val_loss: 0.5523 - val_accuracy: 0.8190
Epoch 5/1000
301/301 [==============================] - 1s 2ms/step - loss: 0.6287 - accuracy: 0.7942 - val_loss: 0.5167 - val_accuracy: 0.8302
Epoch 6/1000
301/301 [==============================] - 1s 2ms/step - loss: 0.6000 - accuracy: 0.8072 - val_loss: 0.5201 - val_accuracy: 0.8402
Epoch 7/1000
301/301 [==============================] - 1s 2ms/step - loss: 0.5691 - accuracy: 0.8150 - val_loss: 0.4684 - val_accuracy:

In [242]:
# Save model
scaler_filename = "./model-server/ml-models/scaler_v2.pkl"  
neural_net_model_file = "./model-server/ml-models/neural_net_v4.h5"  
joblib.dump(final_scaler, scaler_filename)
nn_model.model.save(neural_net_model_file)